In [2]:
import pandas as pd
import numpy as np

import unidecode
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LinearRegression
import sklearn.metrics as sm
from sklearn.metrics import r2_score

In [27]:
fb_df = pd.read_csv('data/Premier-League-2015-2019.csv')


In [4]:
fb_df.sample(10)

,Date,HomeTeam,AwayTeam,outcome_by_HOME,AVERAGE_ODD_WIN,AVERAGE_ODD_DRAW,AVERAGE_ODD_OPPONENT_WIN,RANKINGHOME,RANKING AWAY,LAST_GAME_RHOMETEAM,LAST_GAME_RAWAYTEAM
1252,10/11/2018,Huddersfield,West Ham,D,7.144,6.955,2.449,16,13,1,1
472,29/10/2016,Middlesbrough,Bournemouth,W,8.616,3.064,3.085,20,16,0,-1
424,17/09/2016,Man City,Bournemouth,W,8.010,5.402,10.657,4,16,-1,0
1002,30/01/2018,West Ham,Crystal Palace,D,6.840,3.246,2.600,11,14,0,-1
1059,12/03/2018,Stoke,Man City,L,15.478,7.384,2.585,13,3,0,1
250,13/02/2016,Bournemouth,Stoke,L,7.050,3.187,3.689,20,9,-1,-1
895,29/11/2017,Bournemouth,Burnley,L,5.769,3.207,3.956,9,16,0,-1
341,23/04/2016,Liverpool,Newcastle,D,7.013,4.007,5.488,6,15,-1,0
151,12/12/2015,Crystal Palace,Southampton,W,7.979,3.165,2.882,10,7,-1,-1
454,15/10/2016,Man City,Everton,D,7.215,4.261,6.305,4,11,-1,0


In [5]:
fb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1520 entries, 0 to 1519
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Date                      1520 non-null   object 
 1   HomeTeam                  1520 non-null   object 
 2   AwayTeam                  1520 non-null   object 
 3   outcome_by_HOME           1520 non-null   object 
 4   AVERAGE_ODD_WIN           1520 non-null   float64
 5   AVERAGE_ODD_DRAW          1520 non-null   float64
 6   AVERAGE_ODD_OPPONENT_WIN  1520 non-null   float64
 7   RANKINGHOME               1520 non-null   int64  
 8   RANKING AWAY              1520 non-null   int64  
 9   LAST_GAME_RHOMETEAM       1520 non-null   int64  
 10  LAST_GAME_RAWAYTEAM       1520 non-null   int64  
dtypes: float64(3), int64(4), object(4)
memory usage: 130.8+ KB


Check for duplicates

In [6]:
fb_df.duplicated().sum()

0

Checking for missing values

In [7]:
fb_df.isnull().sum()

Date                        0
HomeTeam                    0
AwayTeam                    0
outcome_by_HOME             0
AVERAGE_ODD_WIN             0
AVERAGE_ODD_DRAW            0
AVERAGE_ODD_OPPONENT_WIN    0
RANKINGHOME                 0
RANKING AWAY                0
LAST_GAME_RHOMETEAM         0
LAST_GAME_RAWAYTEAM         0
dtype: int64

 normalize and standardize column names

In [8]:
fb_df.rename(columns={
    'Date': 'date',
    'HomeTeam': 'home_team',
    'AwayTeam': 'away_team',
    'outcome_by_HOME': 'home_outcome',
    'AVERAGE_ODD_WIN': 'avg_odd_home_win',
    'AVERAGE_ODD_DRAW': 'avg_odd_draw',
    'AVERAGE_ODD_OPPONENT_WIN': 'avg_odd_away_win',
    'RANKINGHOME': 'home_ranking',
    'RANKING AWAY': 'away_ranking',
    'LAST_GAME_RHOMETEAM': 'last_home_result',
    'LAST_GAME_RAWAYTEAM': 'last_away_result'
}, inplace=True)

In [9]:
fb_df.head()

,date,home_team,away_team,home_outcome,avg_odd_home_win,avg_odd_draw,avg_odd_away_win,home_ranking,away_ranking,last_home_result,last_away_result
0,08/08/2015,Bournemouth,Aston Villa,L,7.024,3.303,3.748,20,17,0,0
1,08/08/2015,Chelsea,Swansea,D,6.697,4.338,8.967,1,8,0,0
2,08/08/2015,Everton,Watford,D,6.842,3.514,4.852,11,20,0,0
3,08/08/2015,Leicester,Sunderland,W,7.000,3.207,3.742,14,16,0,0
4,08/08/2015,Man United,Tottenham,W,6.477,3.651,5.318,4,5,0,0


Label encoding

label encode home and away team and make sure a team in Home will have the same ID in Away

In [10]:
all_teams = pd.concat([fb_df['home_team'], fb_df['away_team']]).unique()

team_encoder = LabelEncoder()
team_encoder.fit(all_teams)

fb_df['home_team'] = team_encoder.transform(fb_df['home_team'])
fb_df['away_team'] = team_encoder.transform(fb_df['away_team'])

In [11]:
fb_df.sample(10)

,date,home_team,away_team,home_outcome,avg_odd_home_win,avg_odd_draw,avg_odd_away_win,home_ranking,away_ranking,last_home_result,last_away_result
1231,27/10/2018,9,2,L,6.849,6.905,2.381,20,12,-1,0
1318,22/12/2018,26,24,L,6.230,6.689,2.877,13,14,1,1
434,24/09/2016,16,23,L,9.558,3.520,2.254,20,3,-1,1
647,06/03/2017,26,6,L,10.144,4.295,2.086,7,10,-1,1
464,22/10/2016,12,7,W,8.053,3.267,3.735,1,15,1,1
702,17/04/2017,16,0,L,10.238,4.478,2.075,20,2,0,1
96,25/10/2015,2,23,L,7.875,3.337,2.369,20,5,-1,1
326,16/04/2016,15,1,W,7.249,5.601,13.862,4,17,-1,-1
1157,19/08/2018,4,24,L,7.059,6.939,3.165,7,14,0,1
969,01/01/2018,3,2,D,6.145,3.050,3.252,20,9,0,1


Label encoding for home_outcome

In [12]:
#label encoding for home_outcome
df_label = fb_df
df_label['home_outcome'] = df_label['home_outcome'].map({'L': 0, 'D': 1, 'W': 2})

In [13]:
df_label.sample(10)

,date,home_team,away_team,home_outcome,avg_odd_home_win,avg_odd_draw,avg_odd_away_win,home_ranking,away_ranking,last_home_result,last_away_result
579,04/01/2017,23,6,2,7.861,3.270,2.630,3,10,-1,1
932,16/12/2017,6,19,2,5.594,4.530,8.068,1,8,1,-1
403,27/08/2016,11,15,0,12.562,4.779,2.184,20,5,1,0
248,07/02/2016,2,0,0,8.415,3.478,2.245,20,3,0,0
947,23/12/2017,20,25,2,6.093,3.114,3.326,13,10,-1,-1
936,16/12/2017,24,10,0,5.739,3.467,4.642,17,20,-1,-1
770,19/08/2017,2,24,0,5.987,3.365,3.798,9,17,-1,0
245,06/02/2016,20,8,0,7.473,3.233,2.783,9,11,1,0
1058,11/03/2018,2,23,0,9.903,4.681,2.042,9,2,0,1
149,07/12/2015,8,7,1,7.056,3.301,3.520,11,10,-1,1


One-Hot encode for home_outcome

might be useful if using differnt ml model?

In [14]:
# One-hot encoding for home_outcome
##df_onehot = fb_df
##df_onehot = pd.get_dummies(fb_df, columns=['home_outcome'])

In [15]:
df_label.sample(10)

,date,home_team,away_team,home_outcome,avg_odd_home_win,avg_odd_draw,avg_odd_away_win,home_ranking,away_ranking,last_home_result,last_away_result
124,21/11/2015,19,20,0,6.433,3.596,5.818,7,9,0,1
1515,12/05/2019,13,27,2,6.619,8.266,8.563,4,20,1,1
448,02/10/2016,15,20,1,7.977,4.979,9.229,5,9,1,-1
641,04/03/2017,20,16,2,7.186,3.187,4.167,9,20,1,1
1321,26/12/2018,4,8,0,7.698,7.128,2.218,7,8,-1,-1
43,12/09/2015,15,13,2,6.077,3.248,4.084,4,6,-1,-1
825,30/09/2017,25,24,1,6.382,3.064,3.380,10,17,-1,1
530,13/12/2016,2,12,2,7.657,3.301,3.020,16,1,1,0
1238,28/10/2018,15,8,2,6.419,6.998,4.366,2,8,0,1
63,26/09/2015,17,6,1,9.637,4.180,2.078,15,1,0,1


Checking for outliers

In [16]:
df_label.describe()

,home_team,away_team,home_outcome,avg_odd_home_win,avg_odd_draw,avg_odd_away_win,home_ranking,away_ranking,last_home_result,last_away_result
count,1520.000000,1520.000000,1520.000000,1520.000000,1520.000000,1520.000000,1520.000000,1520.000000,1520.000000,1520.000000
mean,13.687500,13.687500,1.155921,7.576486,4.878582,4.741020,10.650000,10.650000,-0.067763,0.067763
std,8.070483,8.070483,0.859462,1.715519,2.011393,3.807761,5.987578,5.987578,0.852530,0.857918
min,0.000000,0.000000,0.000000,5.324000,2.935000,2.005000,1.000000,1.000000,-1.000000,-1.000000
25%,7.000000,7.000000,0.000000,6.507000,3.290750,2.491500,5.750000,5.750000,-1.000000,-1.000000
50%,14.000000,14.000000,1.000000,7.117500,3.921500,3.285500,10.500000,10.500000,0.000000,0.000000
75%,21.000000,21.000000,2.000000,8.053250,6.617250,5.095000,15.250000,15.250000,1.000000,1.000000
max,27.000000,27.000000,2.000000,18.767000,14.597000,33.015000,20.000000,20.000000,1.000000,1.000000


In [17]:
df_label.shape

(1520, 11)

removing outliers for "avg_odd_home_win", "avg_odd_draw" and "avg_odd_away_win"

In [18]:
def remove_outliers_iqr(df, column):
    initial_count = len(df)
    
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    df_filtered = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    
    removed_count = initial_count - len(df_filtered)
    print(f"{removed_count} rows removed from '{column}' due to outliers.")
    
    return df_filtered

In [19]:
for col in ['avg_odd_home_win', 'avg_odd_draw', 'avg_odd_away_win']:
    df_label = remove_outliers_iqr(df_label, col)

101 rows removed from 'avg_odd_home_win' due to outliers.
6 rows removed from 'avg_odd_draw' due to outliers.
167 rows removed from 'avg_odd_away_win' due to outliers.


In [26]:
df_label 


X = df_label [['avg_odd_home_win', 'avg_odd_draw', 'avg_odd_away_win']]

# Use describe on the selected columns
X.describe()

#print(X)

,avg_odd_home_win,avg_odd_draw,avg_odd_away_win
count,1246.000000,1246.000000,1246.000000
mean,7.274838,4.441469,3.684693
std,1.043937,1.647506,1.604586
min,5.463000,2.935000,2.005000
25%,6.492000,3.250000,2.544750
50%,7.056000,3.535000,3.197000
75%,7.885250,6.383250,4.292000
max,10.355000,9.088000,9.057000


In [21]:
#df_label.to_csv('data/cleaned-premier-league-data.csv', index=False)
